Demo code adapted from [Chapter 11: area chart code](https://github.com/scotthmurray/d3-book/blob/master/chapter_11/06_area_chart.html) of the [d3-book](https://github.com/scotthmurray/d3-book) repo

In [13]:
%%javascript

require.config({
    'paths': {
        d3: 'https://cdnjs.cloudflare.com/ajax/libs/d3/4.5.0/d3'
    }
});

require(["d3" ] , function(d3) {
    element.append(`
    <style type="text/css">
        .area {
            fill: teal;
            stroke: none;
        }

        .line {
            fill: none;
            stroke: teal;
            stroke-width: 0.5;
        }

        .safeLevel {
            stroke: red;
            stroke-dasharray: 2, 3;
        }

        .danger {
            fill: red;
        }

        .dangerLabel {
            font-family: Helvetica, sans-serif;
            font-size: 12px;
            fill: red;
        }
    </style>
    `);

    //Width and height
    var w = 800;
    var h = 300;
    var padding = 40;

    var dataset, xScale, yScale, xAxis, yAxis, area, dangerArea;  //Empty, for now

    //For converting Dates to strings
    var formatTime = d3.timeFormat("%Y");

    //Function for converting CSV values from strings to Dates and numbers
    var rowConverter = function(d) {
        return {
            date: new Date(+d.year, (+d.month - 1)),  //Make a new Date object for each year + month
            average: parseFloat(d.average)  //Convert from string to float
        };
    }

    //Load in data
    d3.csv("mauna_loa_co2_monthly_averages.csv", rowConverter, function(data) {

        var dataset = data;

        //Print data to console as table, for verification
//         console.table(dataset, ["date", "average"]);

        //Create scale functions
        xScale = d3.scaleTime()
                       .domain([
                            d3.min(dataset, function(d) { return d.date; }),
                            d3.max(dataset, function(d) { return d.date; })
                        ])
                       .range([padding, w]);

        yScale = d3.scaleLinear()
                        .domain([
                            d3.min(dataset, function(d) { if (d.average >= 0) return d.average; }) - 10,
                            d3.max(dataset, function(d) { return d.average; })
                        ])
                        .range([h - padding, 0]);

        //Define axes
        xAxis = d3.axisBottom()
                   .scale(xScale)
                   .ticks(10)
                   .tickFormat(formatTime);

        //Define Y axis
        yAxis = d3.axisLeft()
                   .scale(yScale)
                   .ticks(10);

        //Define area generators
        area = d3.area()
                    .defined(function(d) { return d.average >= 0; })
                    .x(function(d) { return xScale(d.date); })
                    .y0(function() { return yScale.range()[0]; })
                    .y1(function(d) { return yScale(d.average); });

        dangerArea = d3.area()
                    .defined(function(d) { return d.average >= 350; })
                    .x(function(d) { return xScale(d.date); })
                    .y0(function() { return yScale(350); })
                    .y1(function(d) { return yScale(d.average); });

        //Create SVG element
        var svg = d3.select(".rendered_html")
                    .append("svg")
                    .attr("width", w)
                    .attr("height", h);

        //Create areas
        svg.append("path")
            .datum(dataset)
            .attr("class", "area")
            .attr("d", area);

        svg.append("path")
            .datum(dataset)
            .attr("class", "area danger")
            .attr("d", dangerArea);

        //Create axes
        svg.append("g")
            .attr("class", "axis")
            .attr("transform", "translate(0," + (h - padding) + ")")
            .call(xAxis);

        svg.append("g")
            .attr("class", "axis")
            .attr("transform", "translate(" + padding + ",0)")
            .call(yAxis);

        //Draw 350 ppm line
        svg.append("line")
            .attr("class", "line safeLevel")
            .attr("x1", padding)
            .attr("x2", w)
            .attr("y1", yScale(350))
            .attr("y2", yScale(350));

        //Label 350 ppm line
        svg.append("text")
            .attr("class", "dangerLabel")
            .attr("x", padding + 20)
            .attr("y", yScale(350) - 7)
            .text("350 ppm “safe” level");


    });

});

<IPython.core.display.Javascript object>